Random Forest Regression Model

In [270]:
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn import tree

In [271]:
data= pd.read_csv("data/raw/Mall_Customers.csv", sep = ",")
data.info()
data.describe()
data = data.drop(["CustomerID"], axis = 1)
column_list = list(data.columns)
print(column_list)
data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   CustomerID      200 non-null    int64 
 1   Sex             200 non-null    object
 2   Age             200 non-null    int64 
 3   Annual Income   200 non-null    int64 
 4   Spending Score  200 non-null    int64 
dtypes: int64(4), object(1)
memory usage: 7.9+ KB
['Sex', 'Age', 'Annual Income', 'Spending Score']


,Sex,Age,Annual Income,Spending Score
0,Male,19,15,39
1,Male,21,15,81
2,Female,20,16,6
3,Female,23,16,77
4,Female,31,17,40


In [272]:
data['Sex'] = data['Sex'].map({'Male': 1, 'Female': 0})

In [273]:
from sklearn.feature_selection import SelectKBest
from sklearn.model_selection import train_test_split

X = data.drop("Spending Score", axis = 1)
y = data["Spending Score"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

selection_model = SelectKBest(k = 7)
selection_model.fit(X_train, y_train)

selected_columns = X_train.columns[selection_model.get_support()]
X_train_sel = pd.DataFrame(selection_model.transform(X_train), columns = selected_columns)
X_test_sel = pd.DataFrame(selection_model.transform(X_test), columns = selected_columns)

X_train_sel.head()

,Sex,Age,Annual Income
0,0,49,54
1,1,32,126
2,0,36,37
3,0,54,28
4,0,40,69


In [274]:
X_train_sel["Spending Score"] = y_train.values
X_test_sel["Spending Score"] = y_test.values
X_train_sel.to_csv("data/train/clean_train.csv", index = False)
X_test_sel.to_csv("data/test/clean_test.csv", index = False)

In [275]:
test_data= pd.read_csv("data/test/clean_test.csv", sep = ",")
train_data= pd.read_csv("data/train/clean_train.csv", sep = ",")

In [276]:
X_train = train_data.drop(["Spending Score"], axis = 1)
y_train = train_data["Spending Score"]
X_test = test_data.drop(["Spending Score"], axis = 1)
y_test = test_data["Spending Score"]

In [277]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(n_estimators = 60, random_state = 42)
model.fit(X_train, y_train)

RandomForestClassifier(n_estimators=60, random_state=42)

In [278]:
from sklearn.metrics import root_mean_squared_error
y_pred = model.predict(X_test)
print("mean squared error:", root_mean_squared_error(y_test, y_pred))


mean squared error: 25.97402548701298


In [279]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
import numpy as np

# Define the RandomForest model
model = RandomForestClassifier(random_state=42)

# Define the hyperparameter space for RandomizedSearchCV
param_distributions = {
    'n_estimators': [50, 100, 200],          # Number of trees in the forest
    'max_depth': [10, 20, None],             # Maximum depth of the trees
    'min_samples_split': [2, 5, 10],         # Minimum number of samples required to split an internal node
    'min_samples_leaf': [1, 2, 4],           # Minimum number of samples required to be at a leaf node
    'bootstrap': [True, False]               # Whether bootstrap samples are used when building trees
}



In [280]:
# Initialize RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=model,
    param_distributions=param_distributions,
    n_iter=10,  # Number of random combinations to try
    scoring='accuracy',
    cv=3,  # 3-fold cross-validation for faster performance
    verbose=1,
    random_state=42,
    n_jobs=-1  # Use all available processors for parallel processing
)


In [281]:
# Fit the RandomizedSearchCV model
random_search.fit(X_train, y_train)

# Print the best parameters and best score
print("Best Parameters:", random_search.best_params_)
print("Best Score:", random_search.best_score_)

# Use the best estimator for predictions
model = random_search.best_estimator_
y_pred = model.predict(X_test)


Fitting 3 folds for each of 10 candidates, totalling 30 fits
Best Parameters: {'n_estimators': 50, 'min_samples_split': 10, 'min_samples_leaf': 4, 'max_depth': 20, 'bootstrap': True}
Best Score: 0.06266014442115071


In [282]:
from sklearn.metrics import root_mean_squared_error
report = root_mean_squared_error(y_test, y_pred)
print(report)

24.86714700161641


In [283]:
from pickle import dump

dump(model, open("models/random_tree_mall.sav", "wb"))

Streamlit

In [284]:
import streamlit as st
from pickle import load
import os

In [285]:
model_path=os.path.join(os.path.dirname(os.path.abspath(models/random_forest_mall.sav)), "models")
model = load(open(model_path, "rb"))

NameError: name 'models' is not defined

Class dictionary for prediction lables

In [ ]:
class_dict = {
    "0": "Iris",
    "": "",
    "": ""
}

Streamlit App Title

st.title("Spending Score")

Input fields for the spending measurements

In [ ]:
val1 = st.number_input(" ()", min_value=0.0, format="%.2f")
val2 = st.number_input(" ()", min_value=0.0, format="%.2f")
val3 = st.number_input(" ()", min_value=0.0, format="%.2f")
val4 = st.number_input(" ()", min_value=0.0, format="%.2f")

When the user clicks the "Predict" button

In [ ]:
if st.button("Predict"):
    data = [[val1, val2, val3, val4]]
    prediction = str(model.predict(data)[0])
    pred_class=class_dict[prediction]
    st.write(f"Prediction: {pred_class}")